In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.0
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=10
Nul=10
Nup=10**(40)
scal=1
umax=5.3
vmax=200
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.5
#dr0v=0.045
dr0v=-1/(4*-.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 2.0
r-: 0.0
rc: 0.0
Number of points for highest iteration is 10600000.0,5300.0X2000
Total number of points is 10600000
Runtime: about 2.26 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
dsigunpv=np.zeros((1,Nv),dtype=datatype)


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)




Using Standard Coordinates Along V


In [6]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)
drunp=np.array([0.0],dtype=datatype)
dsigunp=np.array([0.0],dtype=datatype)
dphinpuf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigunpv=np.empty(Nv-1)*np.nan
drunpv=np.empty(Nv-1)*np.nan
dsigunpvalue=1.0
drunptemp=1.0
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigunp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=1 
TempTolv=1
Eres=False
dru=float(drunp[0])
drv=float(dr0v)
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=M0
    
    
    

    j=0
    drv=1.0
    for j in range(0,Nv-1):
    #while j<Nv-1 and dru<0.0 and drv>0.0:
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunpv[j]=answer[4]
        dsigunpv[j]=answer[5]
        
        #drunptemp=answer[4]
        #dsigunpvalue=answer[5]
        dru=answer[6]
        drv=answer[7]
        exterm3=answer[8]
        #j+=1
            
    drunptemp=np.nanmax(np.abs(drunpv)) 
    dsigunpvalue=np.nanmax(np.abs(dsigunpv)) 
    
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
        
        rnpf[1,:]=rnpf[1,:]*np.nan
        phinpf[1,:]=phinpf[1,:]*np.nan
        signpf[1,:]=signpf[1,:]*np.nan
        massnpf[1,:]=massnpf[1,:]*np.nan
        
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    print(TempTol)
    print(drunptemp)
    print(drv)
    print(dsigunpvalue)
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du) or np.isnan(drunptemp): #or np.isnan(dru) or :
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.100000000000000005551115
1
1.801617804422751
0.13330285563435104
0.6240190477173658
---
0.100000000000000005551115
0.055505668158092269504067
1
1.8937468342606016
0.13325422864141742
0.6718719634262535
---
0.155505668158092275055182
0.052805368801603413952961
1
1.96586753211649
0.13321831693564973
0.7093935982376115
---
0.208311036959695689008143
0.050868127361734349900502
1
2.0415516841711474
0.13318236335655864
0.7488723981610296
---
0.259179164321430066664220
0.048982350422639021891857
1
2.1214957407687822
0.133146129794985
0.7906776096933612
---
0.308161514744069109372759
0.047136554685592843561004
1
2.2059629670575394
0.133109608734714
0.8349526119892264
---
0.355298069429661966811551
0.045331676684213187411565
1
2.295222998410171
0.1330727969633854
0.8818443699353813
---
0.400629746113875140345328
0.043568751301841637801893
1
2.3895613473447206
0.13303569133206453
0.9315084318423472
---
0.444198497415716764269433
0.041848684952625281319172
1
2.4892805

1
74.39662244532647
0.12973704837959749
40.30276683417299
---
1.309513064664668657854918
0.001344147042071557355755
1
78.79179429630476
0.12967008648328407
42.72412010792141
---
1.310857211706740299561602
0.001269167695609768435616
1
83.4513684418016
0.12960244903141643
45.29240144835569
---
1.312126379402350107028496
0.001198302698532011578356
1
88.39140273250736
0.1295341258510025
48.01662392753945
---
1.313324682100882156987609
0.001131331746172452128174
1
93.62894103332066
0.12946510655717247
50.90636058418672
---
1.314456013847054682841531
0.001068045829594633762752
1
99.18207455301895
0.12939538054790287
53.971779733341016
---
1.315524059676649315520081
0.001008246706379828947289
1
105.07000705024231
0.12932493699882122
57.223682541435196
---
1.316532306383029204965851
0.000951746390881862852108
1
111.31312415726889
0.12925376485739115
60.673543015446796
---
1.317484052773911118450201
0.000898366663922889048359
1
117.93306709410824
0.1291818528374612
64.33355056385292
---
1.31838

1
4915.847010719795
0.12241243965022311
2812.506082784561
---
1.332951808087888379006358
0.000020342374321644657043
1
5224.152728965036
0.12225144957810398
2991.846120152106
---
1.332972150462210070287483
0.000019141859970049373456
1
5552.151453197007
0.12208793105387274
3182.8709158038214
---
1.332991292322180054696901
0.000018011036053855949908
1
5901.128959405972
0.12192182388581249
3386.3619099468215
---
1.333009303358233976766201
0.000016945909958569410012
1
6272.457234603688
0.12175306595203728
3603.1544826132276
---
1.333026249268192575669900
0.000015942715312321820332
1
6667.6004466224795
0.12158159312253812
3834.141821854117
---
1.333042191983504931585003
0.000014997899289339647796
1
7088.121343658187
0.12140733917712233
4080.2790823437995
---
1.333057189882794313362524
0.000014108110619391555542
1
7535.688116096509
0.12123023571949809
4342.587857330738
---
1.333071297993413617533065
0.000013270188264081192564
1
8012.081755454796
0.12105021208700557
4622.160988934557
---
1.333

1
478845.13326855487
0.09719352269210463
316058.75055166654
---
1.333286466687761739891016
0.000000208835786462752121
1
514077.2211839226
0.09638774601006181
340757.2394284645
---
1.333286675523548181487854
0.000000194523304825098968
1
552083.6543033958
0.09555294598136364
367556.27608556265
---
1.333286870046852978788365
0.000000181131970165241180
1
593102.7941909075
0.09468751550447868
396654.3529201618
---
1.333287051178823068298129
0.000000168604837103181938
1
637395.8228102693
0.0937897259960444
428271.21042362973
---
1.333287219783660271232861
0.000000156888383044465823
1
685249.2247638847
0.09285771564395025
462650.44993706845
---
1.333287376672043222924913
0.000000145932306650119688
1
736977.5804442944
0.09188947626567182
500062.5175606621
---
1.333287522604349906174548
0.000000135689337984629052
1
792926.715164373
0.09088283857417614
540808.1200109345
---
1.333287658293687893973356
0.000000126115059673919653
1
853477.2568775687
0.08983545562102296
585222.1446160667
---
1.33328

KeyboardInterrupt: 

In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

269


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

34.72433996500013 seconds
0.5787389994166687 minutes
0.009645649990277812 hours
538000 points
15493.455038807619 points per second
Took 1.69 less minutes


conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            